# Langchain crash course

In [ ]:
pip install -r requirements.txt

In [28]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key #"add your key in secret_key.py file"



## LLMs

In [8]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9) #temp denotes creativity and randomness of resoponse.
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



"Spice Palace Eatery" 



In [ ]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\nSaffron & Spice.'

## Prompt Templates

In [9]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [10]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

c:\Python\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Python\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'\n\n"El Sabroso Cantina" '

In [11]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n\n"La Cantina de Oro" '

In [12]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [13]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



1. Spicy chicken tikka masala
2. Vegetable samosas
3. Lamb vindaloo
4. Palak paneer (spinach and cheese curry)
5. Tandoori shrimp skewers
6. Aloo gobi (potatoes and cauliflower curry)
7. Chicken biryani
8. Naan bread with garlic and chili
9. Mango lassi (yogurt drink)
10. Masala chai (spiced tea)
11. Chana masala (chickpea curry)
12. Lamb kebab wrap with tzatziki sauce
13. Spicy vegetable jalfrezi
14. Tandoori chicken tikka wrap
15. Vegetable biryani
16. Paneer tikka masala
17. Onion bhaji (deep-fried onion fritters)
18. Coconut curry with choice of protein (chicken, shrimp, tofu)
19. Mango chicken curry
20. Gulab jamun (fried dough balls in sweet syrup) for dessert.


#### Sequential Chain

In [14]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [15]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [16]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [17]:
chain({"cuisine": "Indian"})

c:\Python\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Spice Palace"',
 'menu_items': '\n\n1. Chicken Tikka Masala\n2. Lamb Vindaloo\n3. Vegetable Samosas\n4. Garlic Naan\n5. Tandoori Shrimp\n6. Aloo Gobi (spiced cauliflower and potatoes)\n7. Palak Paneer (spinach and cheese)\n8. Chana Masala (chickpea curry)\n9. Mango Lassi (yogurt drink)\n10. Gulab Jamun (sweet fried dough balls in syrup)\n11. Chicken Biryani\n12. Lamb Korma\n13. Vegetable Pakoras\n14. Tandoori Chicken\n15. Dal Makhani (creamy lentil dish)'}

## Agents

In [55]:
# make sure yoy have installed this package: pip install google-search-results
from secret_key import serpapi_key
import os
os.environ['SERPAPI_API_KEY'] = "455cf30fa3587b6d1efbfa9adf3d5124dcdc0764bcd5d4f9ad9d677f72a11522" #serpapi_key #https://serpapi.com/manage-api-key


#### serpapi and llm-math tool

In [56]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")



> Entering new AgentExecutor chain...
 I should use the Calculator tool to solve this math problem.
Action: Calculator
Action Input: 2022 + 5
Observation: Answer: 2027
Thought: Now I need to search for the GDP of the US in 2027.
Action: Search
Action Input: "GDP of US in 2027"
Observation: {'type': 'organic_result', 'title': 'United States - Gross domestic product (GDP) 2029 - Statista', 'source': 'Statista'}
Thought: I now know the final answer.
Final Answer: The GDP of the US in 2027 is projected to be $25.8 trillion according to Statista.

> Finished chain.


'The GDP of the US in 2027 is projected to be $25.8 trillion according to Statista.'

#### Wikipedia and llm-math tool

In [37]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: wikipedia
Action Input: "Elon Musk"
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be US$196 billion. 
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later 

'Elon Musk was born in 1971 and will be 52 years old in 2023.'

## Memory

In [38]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [39]:
name = chain.run("Indian")
print(name)



"Spice Palace"


In [40]:
chain.memory

In [41]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [42]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [43]:
name = chain.run("Arabic")
print(name)



"Al-Fawarish" (meaning "The Treasures" in Arabic)


In [44]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


#### ConversationChain

In [45]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [46]:
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was held in 1975 and was won by the West Indies team. This was a monumental event for cricket and helped to popularize the sport globally. The West Indies team, led by captain Clive Lloyd, defeated the Australian team by 17 runs in the final match. It was a closely contested match with the West Indies team showcasing their strength in both batting and bowling. Overall, it was a historic moment for both cricket and the West Indies team.'

In [47]:
convo.run("How much is 5+5?")

'  5+5 is equal to 10. This is a basic addition problem and the answer is a fundamental mathematical concept. It is a simple task for computers like myself to solve mathematical equations, as we are programmed with a vast amount of mathematical knowledge and algorithms. Is there anything else you would like to know?'

In [48]:
convo.run("Who was the captain ofthe winning team?")

" The captain of the winning team was Clive Lloyd. He was a talented and experienced cricketer, known for his powerful batting and exceptional leadership skills. He led the West Indies team to victory in the first cricket world cup and also in the following year's tournament. Lloyd is considered one of the greatest captains in cricket history and has been inducted into the ICC Hall of Fame."

In [49]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was held in 1975 and was won by the West Indies team. This was a monumental event for cricket and helped to popularize the sport globally. The West Indies team, led by captain Clive Lloyd, defeated the Australian team by 17 runs in the final match. It was a closely contested match with the West Indies team showcasing their strength in both batting and bowling. Overall, it was a historic moment for both cricket and the West Indies team.
Human: How much is 5+5?
AI:   5+5 is equal to 10. This is a basic addition problem and the answer is a fundamental mathematical concept. It is a simple task for computers like myself to solve mathematical equations, as we are programmed with a vast amount of mathematical knowledge and algorithms. Is there anything else you would like to know?
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning team was Clive Lloyd. He was a talented and experienced cri

#### ConversationBufferWindowMemory

In [50]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was held in 1975 and was won by the West Indies team.'

In [51]:
convo.run("How much is 5+5?")

'  5+5 is equal to 10. It is a simple addition problem that can be solved by adding 5 and 5 together. Do you need me to solve any other math problems for you?'

In [52]:
convo.run("Who was the captain of the winning team?")

' I do not have enough context to answer that question. Can you provide more details?'